In [1]:
import numpy as np
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier

In [28]:
train_df = pd.read_csv('Train_file/train.csv')
info_df = pd.read_csv('Train_file/train_data_dict.csv')
test_df = pd.read_csv('Test_file/test.csv')
sub_df = pd.read_csv('sample.csv')

In [29]:
train_df.head(10)

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50
5,6,23,a,6,X,2,anesthesia,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,4449.0,11-20
6,7,32,f,9,Y,1,radiotherapy,S,B,3.0,31397,7.0,Emergency,Extreme,2,51-60,6167.0,0-10
7,8,23,a,6,X,4,radiotherapy,Q,F,3.0,31397,7.0,Trauma,Extreme,2,51-60,5571.0,41-50
8,9,1,d,10,Y,2,gynecology,R,B,4.0,31397,7.0,Trauma,Extreme,2,51-60,7223.0,51-60
9,10,10,e,1,X,2,gynecology,S,E,3.0,31397,7.0,Trauma,Extreme,2,51-60,6056.0,31-40


In [30]:
for col in train_df.columns:
    print('{}  :  {}'.format(col,train_df[col].nunique()))

case_id  :  318438
Hospital_code  :  32
Hospital_type_code  :  7
City_Code_Hospital  :  11
Hospital_region_code  :  3
Available Extra Rooms in Hospital  :  18
Department  :  5
Ward_Type  :  6
Ward_Facility_Code  :  6
Bed Grade  :  4
patientid  :  92017
City_Code_Patient  :  37
Type of Admission  :  3
Severity of Illness  :  3
Visitors with Patient  :  28
Age  :  10
Admission_Deposit  :  7300
Stay  :  11


In [31]:
print(train_df.shape)
print(test_df.shape)

(318438, 18)
(137057, 17)


In [32]:
train_df1 = train_df.copy()
test_df1 = test_df.copy()

In [33]:
train_df1.drop(['case_id','patientid'],axis=1,inplace=True)
case_id = test_df1['case_id']
test_df1.drop(['case_id','patientid'],axis=1,inplace=True)

In [34]:
train_df1.fillna(-1,inplace=True)
test_df1.fillna(-1,inplace=True)

In [35]:
cols=['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Hospital_region_code', 'Available Extra Rooms in Hospital',
       'Department','Ward_Facility_Code', 'Bed Grade',
       'City_Code_Patient','Age']

In [36]:
ohe=ce.OneHotEncoder(cols=cols)
train_objs_num = len(train_df1)
dataset = pd.concat(objs=[train_df1, test_df1], axis=0)
dataset = ohe.fit_transform(dataset)
train_df1 = dataset[:train_objs_num]
test_df1 = dataset[train_objs_num:]

test_df1.drop(['Stay'],axis=1,inplace=True)

C:\Users\Raushan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Raushan\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
ward_map = {
    'U':1,
    'P':2,
    'Q':3,
    'R':4,
    'T':5,
    'S':6
}

type_map = {
    'Urgent':1,
    'Emergency':2,
    'Trauma':3
}

severity_map = {
    'Minor':1,
    'Moderate':2,
    'Extreme':3
}

In [38]:
train_df1['Ward_Type'] = train_df1['Ward_Type'].apply(lambda x: ward_map[x])
train_df1['Type of Admission'] = train_df1['Type of Admission'].apply(lambda x: type_map[x])
train_df1['Severity of Illness'] = train_df1['Severity of Illness'].apply(lambda x: severity_map[x])

test_df1['Ward_Type'] = test_df1['Ward_Type'].apply(lambda x: ward_map[x])
test_df1['Type of Admission'] = test_df1['Type of Admission'].apply(lambda x: type_map[x])
test_df1['Severity of Illness'] = test_df1['Severity of Illness'].apply(lambda x: severity_map[x])

C:\Users\Raushan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Raushan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Raushan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [39]:
target = train_df1['Stay']
train_df1.drop(['Stay'],axis=1,inplace=True)

In [40]:
le = ce.OrdinalEncoder()
le.fit(target)
target = le.transform(target)

In [41]:
train_df2 = train_df.copy()
test_df2 = test_df.copy()

In [42]:
train_df2.drop(['case_id'],axis=1,inplace=True)
test_df2.drop(['case_id'],axis=1,inplace=True)

In [43]:
train_df2.fillna(-1,inplace=True)
test_df2.fillna(-1,inplace=True)

In [44]:
col_target = ['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital','patientid','Hospital_region_code','Department','Ward_Facility_Code','patientid','Bed Grade','City_Code_Patient','Age']
rem_col = ['Available Extra Rooms in Hospital','Ward_Type','Type of Admission','Severity of Illness','Visitors with Patient','Admission_Deposit','Stay']

In [45]:
def target_encode_multiclass(X): #X,y are pandas df and series
    y=X['Stay'].astype(str)   #convert to string to onehot encode
    enc=ce.OneHotEncoder().fit(y)
    y_onehot=enc.transform(y)
    class_names=y_onehot.columns  #names of onehot encoded columns
    X_obj=X[col_target] #separate categorical columns
    X=X[rem_col] 
    for class_ in class_names:
      
        enc=ce.TargetEncoder()
        enc.fit(X_obj,y_onehot[class_]) #convert all categorical 
        temp=enc.transform(X_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        X=pd.concat([X,temp],axis=1)    #add to original dataset
      
    return X

In [46]:
train_df2 = target_encode_multiclass(train_df2)

In [47]:
def target_encode_multiclass(X,t): #X,y are pandas df and series
    y=X['Stay'].astype(str)   #convert to string to onehot encode
    enc=ce.OneHotEncoder().fit(y)
    y_onehot=enc.transform(y)
    class_names=y_onehot.columns  #names of onehot encoded columns
    X_obj=X[col_target] #separate categorical columns
    t_obj=t[col_target]
    t=t[['Available Extra Rooms in Hospital','Ward_Type','Type of Admission','Severity of Illness','Visitors with Patient','Admission_Deposit']] 
    for class_ in class_names:
        enc=ce.TargetEncoder()
        enc.fit(X_obj,y_onehot[class_]) #convert all categorical 
        temp=enc.transform(t_obj)       #columns for class_
        temp.columns=[str(x)+'_'+str(class_) for x in temp.columns]
        t=pd.concat([t,temp],axis=1)    #add to original dataset
      
    return t

In [48]:
test_df2 = target_encode_multiclass(train_df,test_df2)

In [49]:
print(train_df2.shape)
print(test_df2.shape)

(318438, 128)
(137057, 127)


In [50]:
train_df2.drop(['Ward_Type','Type of Admission','Severity of Illness','Visitors with Patient','Admission_Deposit','Stay'],axis=1,inplace=True)
test_df2.drop(['Ward_Type','Type of Admission','Severity of Illness','Visitors with Patient','Admission_Deposit'],axis=1,inplace=True)

In [51]:
X_train = pd.concat(objs=[train_df1,train_df2],axis=1)
X_test = pd.concat(objs=[test_df1,test_df2],axis=1)

In [52]:
print(X_train.shape)
print(X_test.shape)

(318438, 262)
(137057, 262)


In [53]:
hgbc = HistGradientBoostingClassifier(learning_rate=0.01,max_iter=800,max_depth=22,min_samples_leaf=28,max_bins=110,max_leaf_nodes=50)

hgbc.fit(X_train,target)
pred = hgbc.predict(X_test)
sub = pd.DataFrame({'case_id':case_id,'Stay':pred})
sub['Stay'] = le.inverse_transform(sub['Stay'])

sub.to_csv('target.csv',index=False)

C:\Users\Raushan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
